In [ ]:
import shutil
import os.path as osp
from pathlib import Path
from natsort import natsorted
from tqdm.notebook import tqdm

# reload notebook automatically after changes to source python files
%load_ext autoreload
%autoreload 2

# change base folder to parent
import os
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
print(os.getcwd())

import numpy as np
from tf_utils import compose_tf_matrix
# from src.utils.colmap.read_write_model import read_images_binary

# for evo ######################################################################
from evo.core import metrics
# from evo.core.units import Unit

from evo.tools import log
log.configure_logging(verbose=False, debug=False, silent=True)

import pprint
import numpy as np

from evo.tools import plot
import matplotlib.pyplot as plt

# temporarily override some package settings
from evo.tools.settings import SETTINGS
SETTINGS.plot_usetex = False

plot.apply_settings(SETTINGS)

from evo.tools import file_interface
import copy
#############################################################################


In [ ]:
scenario_path = Path("/scratch/kumaraditya_gupta/Datasets/TorWIC-SLAM/Jun15/Aisle_CCW_Run_1")

experiment_path = Path("/scratch/rohit_jayanti/indoor-topo-loc/hloc_experiments/torwic_sfm_superpoint_inloc_superpoint+lightglue_netvlad_exhaustive")
model_name = "sfm_superpoint+lightglue_down3"

outputs = Path(f"/scratch/rohit_jayanti/indoor-topo-loc/hloc_experiments/torwic_eval_trajectory/{experiment_path.name}_{model_name}")
if outputs.exists():
    shutil.rmtree(outputs)
outputs.mkdir(parents=True, exist_ok=True)

gt_poses_path = scenario_path / 'traj_gt.txt'
gt_poses = gt_poses_path.read_text().splitlines()
gt_poses_list = np.array([list(map(float, pose.split())) for pose in gt_poses])

DOWNSAMPLE = 3
gt_poses_list = gt_poses_list[::DOWNSAMPLE]

num_poses = len(gt_poses_list)
print(f"Number of poses: {num_poses}")

START_POSE = 0
END_POSE = num_poses

### Computing Ground Truth Image poses and writing to KITTI Format

In [ ]:
# for image_right
qvec_l2c = np.array([0.41406507, -0.6100328, 0.57049433, -0.3618651])
tvec_l2c = np.array([0.07686256, -0.15441064, -0.1026438])
tf_l2c = compose_tf_matrix(qvec_l2c, tvec_l2c, fix_qvec=True)

image_gt_poses_list = []
for i in tqdm.tqdm(range(START_POSE, END_POSE)):   

    # read pose - Ouster Lidar
    qvec_w2l = np.array(gt_poses_list[i][4:8])
    tvec_w2l =  np.array(gt_poses_list[i][1:4])
    tf_w2l= compose_tf_matrix(qvec_w2l, tvec_w2l, fix_qvec=True)

    tf_w2c = tf_w2l @ tf_l2c
    image_gt_poses_list.append(tf_w2c)
    
image_gt_poses_kitti = np.array([pose[:3, :].reshape(-1) for pose in image_gt_poses_list])
gt_poses_file = outputs / 'gt_image_right_poses.txt'

np.savetxt(gt_poses_file, image_gt_poses_kitti, fmt='%.6f')
print(f'> Saved GT camera poses to {gt_poses_file}')

### Extracting Estimated Poses from COLMAP model

In [ ]:
sfm_dir = experiment_path / model_name
assert sfm_dir.exists(), f"Path does not exist: {sfm_dir}"
print(f"Reading COLMAP model from {sfm_dir}")

pred_images = read_images_binary(sfm_dir / 'images.bin')

image_pred_poses_list = []

for i in tqdm.tqdm(range(START_POSE, END_POSE)):   
    pred_image = pred_images[i+1]
    qvec_c2w = pred_image.qvec
    tvec_c2w = pred_image.tvec
    
    tf_c2w = compose_tf_matrix(qvec_c2w, tvec_c2w, fix_qvec=False)
    tf_w2c = np.linalg.inv(tf_c2w)
    image_pred_poses_list.append(tf_w2c)

# UNCOMMENT THIS TO SCALE THE PREDICTED TRAJECTORY positions using GT as reference
# for i in tqdm.tqdm(range(START_POSE+1, END_POSE)):   
#     image_pred_poses_list[i][:3, 3] *= np.linalg.norm(image_gt_poses_list[i][:3, 3]) / np.linalg.norm(image_pred_poses_list[i][:3, 3])


image_pred_poses_kitti = np.array([pose[:3, :].reshape(-1) for pose in image_pred_poses_list])
pred_poses_file = outputs / 'pred_image_right_poses.txt' 

np.savetxt(pred_poses_file, image_pred_poses_kitti, fmt='%.6f')
print(f'> Saved estimated camera poses to {pred_poses_file}')

In [ ]:
assert len(image_gt_poses_kitti) == len(image_pred_poses_kitti), "Number of poses do not match"
print(f"Number of poses: {len(image_gt_poses_kitti)}")

### Evaluating the estimated poses using the EVO toolkit

In [ ]:
# %matplotlib widget
%matplotlib inline

traj_ref = file_interface.read_kitti_poses_file(str(gt_poses_file))
traj_est = file_interface.read_kitti_poses_file(str(pred_poses_file))
print(f"Trajectory ref scale: {traj_ref.scale}")

EYEBALL_SCALE = 8.0

traj_est_aligned = copy.deepcopy(traj_est)
traj_est_aligned.align(traj_ref, correct_scale=True, correct_only_scale=False)
traj_est_aligned.align_origin(traj_ref)
traj_est_aligned.scale(EYEBALL_SCALE)

In [ ]:
fig = plt.figure(figsize=(10, 20))
traj_by_label = {
    "reference": traj_ref, 
    "estimate (not aligned)": traj_est,
    "estimate (aligned)": traj_est_aligned,
    # "estimate (aligned and scaled w/gt)": traj_est_scaled_aligned,
}


plot.trajectories(fig, traj_by_label, plot.PlotMode.xy)
plt.show()

In [ ]:
pose_relation = metrics.PoseRelation.translation_part
use_aligned_trajectories = True

if use_aligned_trajectories:
    data = (traj_ref, traj_est_aligned) 
else:
    data = (traj_ref, traj_est)

ape_metric = metrics.APE(pose_relation)
ape_metric.process_data(data)

ape_stats = ape_metric.get_all_statistics()
pprint.pprint(ape_stats)

# create an x array with the time in seconds corresponding to each pose
seconds_from_start = list(range(START_POSE, END_POSE))

fig = plt.figure()
plot.error_array(fig.gca(), ape_metric.error, x_array=seconds_from_start,
                statistics={s:v for s,v in ape_stats.items() if s != "sse"},
                name="APE", title="APE w.r.t. " + ape_metric.pose_relation.value, xlabel="$t$ (s)")
plt.show()

In [ ]:
plot_mode = plot.PlotMode.xy
fig = plt.figure(figsize=(15, 5))
ax = plot.prepare_axis(fig, plot_mode)
plot.traj(ax, plot_mode, traj_ref, '--', "gray", "reference")
plot.traj_colormap(ax, traj_est_aligned if use_aligned_trajectories else traj_est, ape_metric.error, plot_mode, min_map=ape_stats["min"], max_map=ape_stats["max"])
ax.legend()
plt.show()